In [4]:
#uploading different packages- to remove the one we do not need
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import LeaveOneOut,KFold
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import Imputer
import os
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics 
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import LeaveOneOut,ShuffleSplit
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.cross_validation import StratifiedKFold
from sklearn import svm
from random import seed
seed(123)
rcParams['figure.figsize'] = 12, 4
import json

In [2]:
"""
#set path to imprt and save files from and in
with open('PATHS.json') as json_data:
    paths = json.load(json_data)

#upload data
train = pd.read_csv(paths["TRAIN_PATH"],index_col='Unnamed: 0')
test = pd.read_csv(paths["TEST_PATH"],index_col='Unnamed: 0')
"""

'\n#set path to imprt and save files from and in\nwith open(\'PATHS.json\') as json_data:\n    paths = json.load(json_data)\n\n#upload data\ntrain = pd.read_csv(paths["TRAIN_PATH"],index_col=\'Unnamed: 0\')\ntest = pd.read_csv(paths["TEST_PATH"],index_col=\'Unnamed: 0\')\n'

In [3]:
#set path to imprt and save files from and in
path = 'C:/Users/Yonathan/Desktop/Rafael'

#upload data- no need if you have the clean files
train = pd.read_csv(os.path.join(path,r'train.csv'),index_col='Unnamed: 0')
test = pd.read_csv(os.path.join(path,r'test.csv'),index_col='Unnamed: 0')
#upload data- no need if you have the clean files, and skip to cell # 93 (seperate training and testing data)
final_train = pd.read_csv(os.path.join(path,r'final_train_Rafael.csv'),index_col='df')
final_test = pd.read_csv(os.path.join(path,r'final_test_Rafael.csv'), index_col='df)

In [5]:
#remove labels names from data
train=train.drop('targetName', 1)
#remove unnecessary time cells from data
col_names = list(train)
for name in col_names:
    if name[:4] == "Time":
        train=train.drop(name, 1)
        
col_names = list(test)
for name in col_names:
    if name[:4] == "Time":
        test=test.drop(name, 1)
#train.head()

In [6]:
# build a function vec_size which measures vector magnitude
def vec_size(x,y,z):
    return (np.sqrt(z**2+x**2+y**2))
#create a df vel_mag with the magnitude of the velocity and val_mean which average the velocity of the samples(row)   
def vel(data):    
    vel_magn=pd.DataFrame()
    for i in range(30):
        x = data["velX_"+str(i)]
        y = data["velY_"+str(i)]
        z = data["velZ_"+str(i)]
    #velocity magnitude matrix    
        vel_magn["vel_magnitude_"+str(i)] =vec_size(x,y,z)
    return (vel_magn)

vel_mag_train = vel(train)
vel_mag_test = vel(test)
#mean velocity magnitudevector
vel_mean_train = pd.DataFrame(np.mean(vel_mag_train, axis=1),columns=["vel_mean"])
vel_mean_test= pd.DataFrame(np.mean(vel_mag_test, axis=1),columns=["vel_mean"])


In [7]:
#a function to calculate the acceleration between each step
def acc(data, vel_res):   
    acc_df=pd.DataFrame()
    for i in range(0,29):
        vel1=vel_res["vel_magnitude_"+str(i)]
        vel2=vel_res["vel_magnitude_"+str(i+1)]
        acc_df["accel_"+str(i)]=vel2-vel1
    return (acc_df) 

acc_df_train =acc(train, vel_mag_train)
acc_df_test =acc(test, vel_mag_test)
#mean acc 
acc_mean_train=pd.DataFrame(np.mean(acc_df_train, axis=1),columns=["acc_mean"])
acc_mean_test=pd.DataFrame(np.mean(acc_df_test, axis=1),columns=["acc_mean"])
#print (acc_df)

In [10]:
#angle calculation
def calc_angle(data,pos_index): 
        x_prev = data["posX_"+str(pos_index)] 
        x_curr = data["posX_"+str(pos_index+1)]
        
        y_prev = data["posY_"+str(pos_index)] 
        y_curr = data["posY_"+str(pos_index+1)]
        
        z_prev = data["posZ_"+str(pos_index)] 
        z_curr = data["posZ_"+str(pos_index+1)]
        
        curr_point_vec = [x_curr-x_prev,y_curr-y_prev,z_curr-z_prev]
        curr_point_vec_mag = vec_size(curr_point_vec[0],curr_point_vec[1],curr_point_vec[2])
        curr_point_vec_norm = [curr_point_vec[0]/curr_point_vec_mag,curr_point_vec[1]/curr_point_vec_mag,curr_point_vec[2]/curr_point_vec_mag]
        
        plain_vec =[x_curr-x_prev,y_curr-y_prev,0] 
        plain_vec_mag = vec_size(plain_vec[0],plain_vec[1],0)
        
        plain_vec_norm = [plain_vec[0]/plain_vec_mag,plain_vec[1]/plain_vec_mag,0]
        res = curr_point_vec_norm[0]*plain_vec_norm[0] +curr_point_vec_norm[1]*plain_vec_norm[1] +curr_point_vec_norm[2]* plain_vec_norm[2] 
        angle = np.arccos(res)
        return (angle*180.0/ np.pi)
    
#run it on a whole df
def angle(data, num_of_func = 1):    
    angle_df=pd.DataFrame()
    for i in range(29): #TODO: Why range only until 29?
        #print (train.iloc[:,i])
        if num_of_func==1:
            angle_df["angle_"+str(i)] =calc_angle(data,i)
        else:
            angle_df["angle_"+str(i)] =calc_angle(data,i)
    return angle_df


In [11]:
angle_df_train=angle(train,1)
angle_df_test=angle(test,1)

angle_df_train2 =angle(train,2)
angle_df_test2 =angle(test,2)

#calculate the mean
angle_mean_train = pd.DataFrame(np.mean(angle_df_train, axis=1),columns=["angle_mean"])
angle_mean_test = pd.DataFrame(np.mean(angle_df_test, axis=1),columns=["angle_mean"])
#print (angle_df)

In [12]:
#a df to calculate the change in angles between each step
def angle_che(data,angle_df):
    angle_change_df=pd.DataFrame()
    for i in range(0,28):
        ang1=angle_df["angle_"+str(i)]
        ang2=angle_df["angle_"+str(i+1)]
        angle_change_df["angle_change_"+str(i)]=np.abs(ang2-ang1)
        
    return(angle_change_df)
#print (angle_change_df)


angle_change_df_train = angle_che(train,angle_df_train)
angle_change_df_test = angle_che(test,angle_df_test)

#calculate the mean
angle_change_mean_train = pd.DataFrame(np.mean(angle_change_df_train, axis=1),columns=["angle_change_mean"])
angle_change_mean_test = pd.DataFrame(np.mean(angle_change_df_test, axis=1),columns=["angle_change_mean"])
#print (angle_change_mean)

In [13]:
#count how many time steps each samples has (as non NaN)
def count_time(data):
    res = pd.DataFrame()
    time_vec=[]
    for i in range(0,len(data)):
        sample=data.iloc[i,:]
        time_vec.append((29-sample.isnull().sum()/6)/2)
    res["time_count"] = time_vec
    return res
time_epoch_train = count_time(train)
time_epoch_test = count_time(test) 
#time_epoch

In [14]:
def create_go_up_and_go_down(table):
    up_and_down = pd.DataFrame()
    goes_up = np.zeros(len(table))
    goes_down = np.zeros(len(table))
    for i in range(len(table)):
        if(does_go_up(table.loc[i])):
            goes_up[i] = 1
        if(does_go_down(table.loc[i])):
            goes_down[i] = 1

    up_and_down["goes_up"] = goes_up
    up_and_down["goes_down"] = goes_down
    return up_and_down


def does_go_up(row):
    for i in range(30):
        if(row["velZ_"+str(i)] > 0):
            return True
    return False

def does_go_down(row):
    for i in range(30):
        if(row["velZ_"+str(i)] < 0):
            return True
    return False


#get parabula parameters
def calc_S(row):
    S = []
    for i in range(30):
        if(np.isnan(row["posX_"+str(i)])):
            break
        S.append(((row["posX_"+str(i)])**2 + (row["posY_"+str(i)])**2)**0.5)
    return S

def calc_parabola_params(table):
    res = pd.DataFrame()
    params = np.array([[0 for i in range(len(table))] for j in range(3)])
    for i in range(len(table)):
        row = table.loc[i]
        S = calc_S(row)
        current_params = np.polyfit(S, [row["posZ_"+str(i)] for i in range(len(S))],2)
        cur_f = np.multiply(current_params[0],(np.power(S,2))) + np.multiply(S,current_params[1]) + current_params[2]
        '''
        if(i%1000 == 0):
            plt.plot(S, [row["posZ_"+str(i)] for i in range(len(S))])
            plt.plot(S, cur_f)
            plt.savefig("images/yair_" + str(i) + ".png")
            plt.clf()
            print("iteration " + str(i))
        '''
        params[0][i], params[1][i], params[2][i] = current_params[0], current_params[1], current_params[2]
        res["parabola_parameter_a"] = params[0]
        res["parabola_parameter_b"] = params[1]
        res["parabola_parameter_c"] = params[2]
    return res


In [41]:
#get goes_up and goes_down features:
up_and_down_train = create_go_up_and_go_down(train)
up_and_down_test = create_go_up_and_go_down(test)

In [77]:
up_and_down_test = create_go_up_and_go_down(test)

In [42]:
#get parbola parameters for each features
parabola_params_train = calc_parabola_params(train)
parabola_params_test = calc_parabola_params(test)

In [17]:
#create new variable to store the data
new_train =train.copy(deep=True)
new_test =test.copy(deep=True)      

In [18]:
#remove Position columns from data
col_names=list(new_train)
for name in col_names:
    if str(name)[:3] == "pos":
        new_train=new_train.drop(name, 1)

col_names=list(new_test)
for name in col_names:
    if  str(name)[:3] == "pos":
        new_test=new_test.drop(name, 1)  

In [57]:
pca_train = pd.read_csv(os.path.join(path,r'PCA2dim.csv'),header=None)
pca_test = pd.read_csv(os.path.join(path,r'PCA2dimUnlabeled.csv'),header=None)

In [74]:
pca_train = pca_train.iloc[:,0:2]
pca_test = pca_test.iloc[:,0:2]

In [91]:
#create a vector of all the matrices and array which we which to add to the data

df_add_vec_train=[new_train,vel_mag_train,
                  vel_mean_train,acc_df_train,
                  acc_mean_train,angle_df_train,
                  angle_mean_train,angle_change_df_train,
                  angle_change_mean_train,
                  time_epoch_train, up_and_down_train, 
                  parabola_params_train,pca_train]

df_add_vec_test=[new_test,vel_mag_test,
                  vel_mean_test,acc_df_test,
                  acc_mean_test,angle_df_test,
                  angle_mean_test,angle_change_df_test,
                  angle_change_mean_test,
                  time_epoch_test, up_and_down_test, 
                  parabola_params_test,pca_test]

#merge them to the data
final_train = pd.concat(df_add_vec_train, axis=1)
final_test = pd.concat(df_add_vec_test, axis=1)


In [40]:
pd.set_option('display.max_columns', 500)
#final_test

,velX_0,velY_0,velZ_0,velX_1,velY_1,velZ_1,velX_2,velY_2,velZ_2,velX_3,velY_3,velZ_3,velX_4,velY_4,velZ_4,velX_5,velY_5,velZ_5,velX_6,velY_6,velZ_6,velX_7,velY_7,velZ_7,velX_8,velY_8,velZ_8,velX_9,velY_9,velZ_9,velX_10,velY_10,velZ_10,velX_11,velY_11,velZ_11,velX_12,velY_12,velZ_12,velX_13,velY_13,velZ_13,velX_14,velY_14,velZ_14,velX_15,velY_15,velZ_15,velX_16,velY_16,velZ_16,velX_17,velY_17,velZ_17,velX_18,velY_18,velZ_18,velX_19,velY_19,velZ_19,velX_20,velY_20,velZ_20,velX_21,velY_21,velZ_21,velX_22,velY_22,velZ_22,velX_23,velY_23,velZ_23,velX_24,velY_24,velZ_24,velX_25,velY_25,velZ_25,velX_26,velY_26,velZ_26,velX_27,velY_27,velZ_27,velX_28,velY_28,velZ_28,velX_29,velY_29,velZ_29,vel_magnitude_0,vel_magnitude_1,vel_magnitude_2,vel_magnitude_3,vel_magnitude_4,vel_magnitude_5,vel_magnitude_6,vel_magnitude_7,vel_magnitude_8,vel_magnitude_9,vel_magnitude_10,vel_magnitude_11,vel_magnitude_12,vel_magnitude_13,vel_magnitude_14,vel_magnitude_15,vel_magnitude_16,vel_magnitude_17,vel_magnitude_18,vel_magnitude_19,vel_magnitude_20,vel_magnitude_21,vel_magnitude_22,vel_magnitude_23,vel_magnitude_24,vel_magnitude_25,vel_magnitude_26,vel_magnitude_27,vel_magnitude_28,vel_magnitude_29,vel_mean,accel_0,accel_1,accel_2,accel_3,accel_4,accel_5,accel_6,accel_7,accel_8,accel_9,accel_10,accel_11,accel_12,accel_13,accel_14,accel_15,accel_16,accel_17,accel_18,accel_19,accel_20,accel_21,accel_22,accel_23,accel_24,accel_25,accel_26,accel_27,accel_28,acc_mean,angle_0,angle_1,angle_2,angle_3,angle_4,angle_5,angle_6,angle_7,angle_8,angle_9,angle_10,angle_11,angle_12,angle_13,angle_14,angle_15,angle_16,angle_17,angle_18,angle_19,angle_20,angle_21,angle_22,angle_23,angle_24,angle_25,angle_26,angle_27,angle_28,angle_mean,angle_change_0,angle_change_1,angle_change_2,angle_change_3,angle_change_4,angle_change_5,angle_change_6,angle_change_7,angle_change_8,angle_change_9,angle_change_10,angle_change_11,angle_change_12,angle_change_13,angle_change_14,angle_change_15,angle_change_16,angle_change_17,angle_change_18,angle_change_19,angle_change_20,angle_change_21,angle_change_22,angle_change_23,angle_change_24,angle_change_25,angle_change_26,angle_change_27,angle_change_mean,time_count
0,490.997969,4.637485,132.914458,488.261801,-2.136002,117.620050,483.106061,5.479289,116.442066,483.122824,2.808065,113.468019,480.161323,-4.064276,108.185406,471.917086,-4.138765,103.793492,472.635204,2.932039,94.646230,467.737392,-0.827624,91.416252,458.388612,-6.936814,86.344612,464.235532,-1.557159,79.106792,461.841977,0.153070,75.017838,454.842926,3.599895,67.131712,449.063760,-4.561839,65.191039,447.468983,-0.650709,58.239023,446.734403,-3.391953,58.122970,439.769632,-2.641317,52.061490,443.497874,3.513652,44.790133,440.457461,2.111095,43.171497,433.144560,2.102636,32.616496,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,508.691227,502.233636,496.971069,496.276676,492.214889,483.214191,482.027532,476.587750,466.501480,470.929866,467.894979,459.784420,453.793942,451.243502,450.512388,442.848399,445.767727,442.573169,434.375951,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,469.707515,-6.457591,-5.262567,-0.694393,-4.061788,-9.000697,-1.186659,-5.439781,-10.086271,4.428386,-3.034886,-8.110559,-5.990478,-2.550440,-0.731114,-7.663989,2.919328,-3.194558,-8.197218,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-4.128626,15.005431,13.192814,13.834092,13.971455,13.085651,9.065947,12.866172,10.795011,9.746151,9.367582,7.285679,9.555707,7.781421,8.196859,5.916160,6.482107,4.194349,5.892446,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.790835,1.812617,0.641278,0.137363,0.885804,4.0197

In [83]:
#keep the clean data in CSV files
final_train.to_csv(os.path.join(path,r'final_train_Rafael.csv'), na_rep='NaN',header=True,index=True)
final_test.to_csv(os.path.join(path,r'final_test_Rafael.csv'), na_rep='NaN',header=True,index=True)

In [93]:
#seperatre training data into features and labels
Y =pd.DataFrame(final_train['class'])
X =final_train.drop('class', axis=1)

In [94]:
#split data to training and testinf sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=7)
y_train,y_test=np.ravel(y_train),np.ravel(y_test)

In [98]:
#or
ss = ShuffleSplit(n_splits=5,random_state=123345,test_size=0.2)
for train_index, test_index in ss.split(X_gbdt,np.ravel(y_gbdt)): 
    X_train , X_test = X.loc[train_index,:] , X.loc[test_index,:]
    y_train , y_test = Y.loc[train_index] , Y.loc[test_index]

In [24]:
"""
only for calibration


xgb_model = xgb.XGBClassifier()

#brute force scan for all parameters, here are the tricks
#usually max_depth is 6,7,8
#learning rate is around 0.05, but small changes may make big diff
#tuning min_child_weight subsample colsample_bytree can have 
#much fun of fighting against overfit 
#n_estimators is how many round of boosting
#finally, ensemble xgboost with multiple seeds may reduce variance
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['multi:softmax'],
              'learning_rate': [0.05], #so called `eta` value
              'max_depth': [6],
              'min_child_weight': [11],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [10], #number of trees, change it to 1000 for better results
              'seed': [1337]}


clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=StratifiedKFold(y_train,n_folds=5, shuffle=True), 
                   scoring='f1_macro',
                   verbose=10, refit=True)

clf.fit(X_train, y_train)

#trust your CV!
best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
print('F1 score:', score)
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))
    
"""

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:   55.5s remaining:  1.4min
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed:  1.3min remaining:   53.4s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:  1.6min remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:  1.6min finished


F1 score: 0.29873543772861627
colsample_bytree: 0.7
learning_rate: 0.05
max_depth: 6
min_child_weight: 11
n_estimators: 10
nthread: 4
objective: 'multi:softmax'
seed: 1337
silent: 1
subsample: 0.8


In [101]:
# a very simple XGboost model
xg_train = xgb.DMatrix(X_train, label=y_train)
xg_test = xgb.DMatrix(X_test, label=y_test)
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
#param['eta'] = 0.1
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 26
param['min_child_weight']=11
param['gamma']=0
param['subsample']=0.8
param['scale_pos_weight']=1
param['colsample_bytree']=0.7
param['learning_rate'] =0.05
param['n_estimators']=10 #number of trees, change it to 1000 for better results
param['seed']=123

watchlist = [ (xg_train,'train'), (xg_test, 'test') ]
num_round = 150 #to increase to 400
bst = xgb.train(param, xg_train, num_round, watchlist )
# get prediction
pred = np.int_(bst.predict( xg_test ))

[0]	train-merror:0.691251	test-merror:0.728696
[1]	train-merror:0.644069	test-merror:0.695826
[2]	train-merror:0.621369	test-merror:0.678087
[3]	train-merror:0.589189	test-merror:0.663826
[4]	train-merror:0.571882	test-merror:0.654957
[5]	train-merror:0.566664	test-merror:0.64713
[6]	train-merror:0.555357	test-merror:0.641565
[7]	train-merror:0.541746	test-merror:0.640522
[8]	train-merror:0.538268	test-merror:0.638957
[9]	train-merror:0.533528	test-merror:0.636348
[10]	train-merror:0.526744	test-merror:0.628522
[11]	train-merror:0.520699	test-merror:0.630609
[12]	train-merror:0.514742	test-merror:0.625739
[13]	train-merror:0.511089	test-merror:0.62487
[14]	train-merror:0.503479	test-merror:0.617739
[15]	train-merror:0.499087	test-merror:0.616696
[16]	train-merror:0.494086	test-merror:0.615826
[17]	train-merror:0.489216	test-merror:0.612
[18]	train-merror:0.484302	test-merror:0.609913
[19]	train-merror:0.480388	test-merror:0.60887
[20]	train-merror:0.477692	test-merror:0.607304
[21]	tra

In [102]:
#calculation
f1 = f1_score(y_test, pred,average='macro')
print("F1: %.2f%%" % (f1 * 100.0))

F1: 43.06%


In [ ]:
""" 
not in use

#A simple Gradient boosting model
train_gbdt=final_train.replace([np.inf],[np.nan])
train_gbdt.fillna(0,inplace=True)
y_gbdt=np.ravel(pd.DataFrame(train_gbdt['class']))
X_gbdt=train_gbdt.drop('class', axis=1)

ss = ShuffleSplit(n_splits=5,random_state=123345,test_size=0.2)
for train_index, test_index in ss.split(X_gbdt,np.ravel(y_gbdt)): 
    X_train , X_test = X_gbdt.loc[train_index,:] , X_gbdt.loc[test_index,:]
    y_train , y_test = y_gbdt.loc[train_index] , y_gbdt.loc[test_index]

gbdt = GradientBoostingClassifier(max_depth=5,subsample=0.8,n_estimators=30)
gbdt.fit(X_train,y_train)
pred = gbdt.predict(X_test)
print (classification_report(pred,y_test))
"""

In [ ]:
#pred

In [ ]:
#y_test

In [103]:
#anomaly detection
# use the same dataset
tr_data = X
tr_data.fillna(0,inplace=True)

In [ ]:
clf = svm.OneClassSVM(nu=0.05, kernel="rbf", gamma=0.1, verbose=True)
clf.fit(tr_data)

[LibSVM]

In [82]:
final_test.fillna(0,inplace=True)
pred = clf.predict(final_test)

# inliers are labeled 1, outliers are labeled -1
abnormal = tr_data[pred == -1]

ValueError: X.shape[1] = 218 should be equal to 216, the number of features at training time

In [ ]:
submission_results = bst.predict(final_test)[:,1]

In [ ]:
submission_results[abnormal]=26

In [ ]:
submission_results.to_csv(os.path.join(path,r'Rafael_submission1.csv'),header=True,index=True, index_label='id')